In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\petersunny.markappa\\_00_Peter_Practice_Projects\\_02_In_Progress\\_02_ML_Projects_Krish_Naik_Course_list\\02_DL_MLOPS_DVC_Azure_AWS\\DL_Chicken_Disease_Classification_AWS_AZURE_DVC'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from ImageClassification.constants import *
from ImageClassification.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH):
        self.config_filepath = config_filepath
        self.params_filepath = params_filepath

        self.config = read_yaml(self.config_filepath)
        self.params = read_yaml(self.params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.perpare_callbacks

        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=(config.checkpoint_model_filepath)
        )

        return prepare_callback_config


In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [8]:
class PrepareCallback:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime('%Y-%M-%D-%H-%M-%S')
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_logs_at_{timestamp}'
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = str(self.config.checkpoint_model_filepath),
            save_best_only=True, save_weights_only=False)
    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [9]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    for callback in callback_list:
        print(callback)
except Exception as e:
    raise e

YAML content: {'artifacts_root': 'artifacts', 'data_root_path': '', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion', 'train_dir': '', 'validation_dir': '', 'test_dir': ''}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'update_base_model_path': 'artifacts/prepare_base_model/base_model_update.h5'}, 'perpare_callbacks': {'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.keras'}}
[2024-05-23 21:22:04,048: INFO: common: YAML file: C:\Users\petersunny.markappa\_00_Peter_Practice_Projects\_02_In_Progress\_02_ML_Projects_Krish_Naik_Course_list\02_DL_MLOPS_D